In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def compute_hn(x, a, b, w, h):
    """
    Vectorized version of the piecewise function:
    
        f(x) = 0 for x in [0, (a + b - w)/2],
               linear increase from 0 to h on [(a + b - w)/2, (a + b)/2],
               linear decrease from h to 0 on [(a + b)/2, (a + b + w)/2],
               0 for x in [(a + b + w)/2, 1].
    
    Parameters
    ----------
    x : array_like
        Points at which f(x) is evaluated (values should lie in [0,1]).
    a, b, w, h : float
        Parameters that define the piecewise function.
    
    Returns
    -------
    y : ndarray
        The values of f(x) for each element in x.
    """
    # Convert x to a numpy array if it's not already
    x = np.asarray(x)

    left_increase  = (a + b - w) / 2.0  # start of the "ramp up"
    mid_point      = (a + b) / 2.0      # peak location
    right_decrease = (a + b + w) / 2.0  # end of the "ramp down"

    # Slopes for linear segments
    slope_up   = 2.0 * h / w   # from 0 to h
    slope_down = -2.0 * h / w  # from h to 0

    # We will define our piecewise conditions in NumPy-friendly form:
    # region 1: x <= left_increase              ->  f(x) = 0
    # region 2: left_increase < x <= mid_point  ->  f(x) = slope_up * (x - left_increase)
    # region 3: mid_point < x <= right_decrease ->  f(x) = h + slope_down * (x - mid_point)
    # region 4: x > right_decrease             ->  f(x) = 0
    
    cond_list = [
        (x <= left_increase),
        (x > left_increase) & (x <= mid_point),
        (x > mid_point) & (x <= right_decrease),
        (x > right_decrease)
    ]
    
    # Corresponding expressions for each region
    func_list = [
        0,
        slope_up * (x - left_increase), 
        h + slope_down * (x - mid_point),
        0
    ]
    
    # Use np.select to evaluate these piecewise
    y = np.select(condlist=cond_list, choicelist=func_list, default=0)
    
    return(y, [[a,(a+b-w)/2],[(a+b+w)/2,b]])



In [ ]:
# Example parameters
a, b, w, h = 0.2, 0.6, 0.3, 1.0

# Create a grid of x-values in [0,1]
x_vals = np.linspace(0, 1, 500)

# Compute the piecewise function for each x (vectorized call)
f_vals, intervals = compute_hn(x_vals, a, b, w, h)

# Plot
plt.figure(figsize=(8, 4))
plt.plot(x_vals, f_vals, label="h(x)")
plt.title("Function with Cantor Third Cumulative Areas")
plt.xlabel("x")
plt.ylabel("f(x)")
plt.ylim([-0.1, 1.1*h])  # add a bit of padding for visibility
plt.grid(False)
plt.legend()
plt.show()

print(intervals)

\begin{gathered}
A_n=\left\{\left.\sum_{k=1}^n r_k \cdot \frac{3}{5} \cdot\left(\frac{2}{5}\right)^{k-1} \right\rvert\, r_k \in\{0,1\}\right\} \\
B_n=\left\{\left.\sum_{k=1}^n r_k \cdot \frac{3}{5} \cdot\left(\frac{2}{5}\right)^{k-1}+\left(\frac{2}{5}\right)^n \right\rvert\, r_k \in\{0,1\}\right\}
\end{gathered}

$A_n$ is lower end of each non spike/flat interval interval, $B_n$ is upper end

In [ ]:
def compute_h(x_vals,n_upper):

    n = 1

    intervals_new = [[0,1]]

    f = np.zeros(1000)

    while n <= n_upper:

        intervals_old = intervals_new
        intervals_new = []

        hn = 4*(5/6)**n
        wn = 2/(hn*3**n)

        for interval in intervals_old:

            a = interval[0]
            b = interval[1]

            f_current, intervals = compute_hn(x_vals, a, b, wn, hn)

            for interval2 in intervals:

                intervals_new.append(interval2)

            f = f + f_current

        n = n + 1

    return f


x_vals = np.linspace(0, 1, 1000)
h_vals = compute_h(x_vals, 14)


In [ ]:

# Plot
plt.figure(figsize=(8, 4))
plt.plot(x_vals, h_vals, label="h(x)")
plt.title("Function with Cantor Third Cumulative Areas")
plt.ylabel("h(x)")
plt.xlabel("x")
# plt.ylim([-0.1, 1.1*h])  # add a bit of padding for visibility
plt.grid(False)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import cumtrapz

def plot_integral_of_f(x_vals, f_vals):
    """
    Given arrays of x-values and the corresponding f(x)-values,
    compute and plot the cumulative integral:
    
        F(x) = ∫ f(t) dt from t=0 to t=x.
    
    Parameters
    ----------
    x_vals : array_like
        Monotonically increasing array of x-values.
    f_vals : array_like
        Corresponding values of f(x).
    """
    # Ensure x_vals and f_vals are numpy arrays
    x_vals = np.asarray(x_vals)
    f_vals = np.asarray(f_vals)
    
    # Compute the cumulative integral using scipy.integrate.cumtrapz
    F_vals = cumtrapz(f_vals, x_vals, initial=0)

    # Normalize the integral
    F_vals = F_vals 

    # Return result
    return(F_vals)



# Plot the integral of f(x) = 2x from 0 to x
H_vals = plot_integral_of_f(x_vals, h_vals)


    # Plot the integral F(x)
    plt.figure(figsize=(8, 4))
    plt.plot(x_vals, F_vals, label="f(x) = ∫ h(t) dt")
    plt.title("Function with Cantor Third Critical Values")
    plt.xlabel("x")
    plt.ylabel("f(x) = ∫ h(t) dt")
    plt.grid(False)
    plt.show()


In [ ]:
F_new = np.concatenate([H_vals[::-1], H_vals])
x_vals_new = np.concatenate([-x_vals[::-1], x_vals])

F_new.shape

In [ ]:
import plotly.graph_objects as go

# 4) Create 2D grid (X, Y) and compute Z = F(x) + F(y)
X, Y = np.meshgrid(x_vals_new, x_vals_new, indexing='xy')
Z = np.zeros_like(X)

num_points = len(x_vals_new)


# # Define a transformation matrix, for example a shear transformation
# a, b = 1, 0.5  # coefficients for shear in the X-direction
# c, d = 0.5, 1  # coefficients for shear in the Y-direction

# # Apply the linear transformation
# X = a * X + b * Y
# Y = c * X + d * Y


for i in range(num_points):
    for j in range(num_points):
        Z[i, j] = (F_new[i] + F_new[j])/2


# Theta = np.array(X)
# Psi = np.array(Y)
# R = np.array(Z)

# # Convert spherical coordinates to Cartesian coordinates
# X = R * np.sin(Theta) * np.cos(Psi)
# Y = R * np.sin(Theta) * np.sin(Psi)
# Z = R * np.cos(Theta)

# print('tmp2',X,Y,Z.dtype)

# 5) Use Plotly to make a 3D surface plot
fig = go.Figure(data=[
    go.Surface(
        x=X,        # x-coordinates
        y=Y,        # y-coordinates
        z=Z,        # z-values (F(x)+F(y))
        colorscale='Viridis',
    )
])

# Update layout for better presentation
fig.update_layout(
    title="3D Surface With Critical Values [0,1]",
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='F(x,y,z) = f(x) + f(y) - z = 0'
    ),
    autosize=True,
    width=800,
    height=700
)

fig.show()

In [ ]:
import plotly.graph_objects as go

# 4) Create 2D grid (X, Y) and compute Z = F(x) + F(y)
X, Y = np.meshgrid(x_vals_new*np.pi/2, x_vals_new*np.pi/2, indexing='xy')
Z = np.zeros_like(X)

num_points = len(x_vals_new)


# # Define a transformation matrix, for example a shear transformation
# a, b = 1, 0.5  # coefficients for shear in the X-direction
# c, d = 0.5, 1  # coefficients for shear in the Y-direction

# # Apply the linear transformation
# X = a * X + b * Y
# Y = c * X + d * Y

print('tmp1',X,Y,Z.dtype)

for i in range(num_points):
    for j in range(num_points):
        Z[i, j] = (F_new[i] + F_new[j])/2


X_original = np.array(X)
Y_original = np.array(Y)
Z_original = np.array(Z)

# Change zero values to 1 to avoid division by zero
M = Z_original == 0
Z_original[M] = 1


X = X_original*np.sqrt(X_original**2+Y_original**2+Z_original**2)/Z_original
Y = Y_original*np.sqrt(X_original**2+Y_original**2+Z_original**2)/Z_original
Z = np.sqrt(X_original**2+Y_original**2+Z_original**2)

# Add back the zeros
Z[M] = 0

# Remove

# print('tmp2',X,Y,Z.dtype)

# 5) Use Plotly to make a 3D surface plot
fig = go.Figure(data=[
    go.Surface(
        x=X,        # x-coordinates
        y=Y,        # y-coordinates
        z=Z,        # z-values (F(x)+F(y))
        colorscale='Viridis',
    )
])

# Update layout for better presentation
fig.update_layout(
    title="3D Surface Tangent to Every Sphere of Radius in [0,1]",
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='F(T(x,y,z))=0'
    ),
    autosize=True,
    width=800,
    height=700
)

fig.show()

In [ ]:
x_vals.shape

In [ ]:
import numpy as np

def compute_g(x_vals, n):
    x_vals = np.array(x_vals)
    total = np.zeros_like(x_vals)

    for i in range(1, n + 1):
        a_n = (2 / 5) ** i
        b_n = (3 / 5) * (2 / 5) ** (i - 1)
        h_n = (1 / 2) * (1 / 3) ** i / (1 - b_n) / i

        g_n = np.zeros_like(x_vals)

        # Ramp section: (a_n, 0) to (b_n, h_n)
        ramp_mask = (x_vals >= a_n) & (x_vals < b_n)
        g_n[ramp_mask] = h_n * (x_vals[ramp_mask] - a_n) / (b_n - a_n)

        # Horizontal section: (b_n, h_n) to (1, h_n)
        horiz_mask = (x_vals >= b_n) & (x_vals <= 1)
        g_n[horiz_mask] = h_n

        total += g_n

    return total


In [ ]:
import matplotlib.pyplot as plt

x_vals = np.linspace(0, 1, 1000)
g_vals = compute_g(x_vals, n=19)

plt.plot(x_vals, g_vals)
plt.title("g(x) for n=10")
plt.xlabel("x")
plt.ylabel("g(x)")
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example x range
x_vals = np.linspace(0, 1, 1000)

# Compute both functions
g_vals = compute_g(x_vals, n=10)     # Replace 5 with the desired n
          

# Plot both
plt.plot(x_vals, h_vals, label="f(x)", linestyle='--')   # Dashed line for f
plt.plot(x_vals, g_vals, label="g(x)", linewidth=2)      # Solid line for g

plt.title("Comparison of f(x) and g(x)")
plt.xlabel("x")
plt.ylabel("Function value")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
H_vals = plot_integral_of_f(x_vals, h_vals)
G_vals = plot_integral_of_f(x_vals, g_vals)*x_vals


In [ ]:

# Plot both
plt.plot(x_vals, H_vals, label="H(x)", linestyle='--')   # Dashed line for f
plt.plot(x_vals, G_vals, label="G(x)", linewidth=2)      # Solid line for g

plt.title("Comparison of H(x) and G(x)")
plt.xlabel("x")
plt.ylabel("Function value")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np

# Assume G_vals and x_vals are already defined
# For example:
# x_vals = np.linspace(0, 1, 1000)
# G_vals = G(x_vals)

def s(x):
    x = np.asarray(x)
    s_vals = np.zeros_like(x)

    # For 0 < x < 1
    mask_middle = (x > 0) & (x < 1)
    x_mid = x[mask_middle]
    s_vals[mask_middle] = x_mid**2 / (x_mid**2 + (1 - x_mid)**2)

    # For x >= 1
    mask_high = x >= 1
    s_vals[mask_high] = 1

    # x <= 0 stays 0 by default
    return s_vals

def compute_N(x_vals, G_vals, a, c):
    # Compute transformed argument to s
    s_input = (x_vals - 1) / a + 1
    s_vals = s(s_input)
    
    # Now compute N(x)
    N_vals = s_vals * c + (1 - s_vals) * G_vals
    return N_vals

# Example usage:
a = 0.5
c = 0.5
N_vals = compute_N(x_vals, G_vals, a, c)


In [ ]:

# Plot both
plt.plot(x_vals, H_vals, label="H(x)", linestyle='--')   # Dashed line for f
plt.plot(x_vals, N_vals, label="G(x)", linewidth=2)      # Solid line for g

plt.title("Comparison of H(x) and G(x)")
plt.xlabel("x")
plt.ylabel("Function value")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
z_vals_new = np.linspace(0, 1, 1000)
x_vals_new = np.concatenate([-x_vals[::-1], x_vals])
N_vals_new = np.concatenate([N_vals[::-1], N_vals])
H_vals_new = np.concatenate([H_vals[::-1], H_vals])





In [ ]:
import numpy as np
import plotly.graph_objects as go

def compute_surface_f(x_vals, z_vals, N_vals, H_vals):
    """
    Computes the surface f(x, z) based on:
    - x_vals, z_vals: 1D arrays of x and z
    - N_vals, H_vals: 1D arrays of N(x) and H(x), same length as x_vals
    Returns:
    - f_vals: 2D array of shape (len(x_vals), len(z_vals))
    """
    X, Z = np.meshgrid(x_vals, z_vals, indexing='ij')
    N_grid = N_vals[:, np.newaxis]
    H_grid = H_vals[:, np.newaxis]

    with np.errstate(divide='ignore', invalid='ignore'):
        R = np.where(H_grid != 0, N_grid / H_grid, 0.0)

    term1 = - (R ** 3) * (Z - H_grid) * Z ** 2
    indicator = (Z > H_grid).astype(float)
    term2 = - indicator * 0.1*(Z - H_grid) ** 2

    f_vals = term1 + term2
    return f_vals

def plot_surface_f(x_vals, z_vals, f_vals):
    """
    Plots the surface f(x, z) using Plotly.
    """
    fig = go.Figure(data=[go.Surface(
        x=x_vals,  # x-axis
        y=z_vals,  # y-axis (will appear as vertical in surface)
        z=f_vals.T,  # Transpose to match plotly's (x,y,z) orientation
        colorscale='Viridis',
        showscale=True
    )])

    fig.update_layout(
        title='Surface Plot of f(x, z)',
        scene=dict(
            xaxis_title='x',
            yaxis_title='z',
            zaxis_title='f(x, z)'
        ),
        autosize=True
    )

    fig.show()


In [ ]:
# Compute surface
f_vals_new = compute_surface_f(x_vals_new, z_vals_new, N_vals_new, H_vals_new)

# Plot it
# plot_surface_f(x_vals_new, z_vals_new, f_vals_new)


In [ ]:
import numpy as np
import plotly.graph_objects as go

def plot_polar_surface(x_vals, z_vals, f_vals):
    """
    Plots f(r, theta) as a 3D surface using polar coordinates.
    x_vals: radial coordinates (r)
    z_vals: angular coordinates (theta, in radians)
    f_vals: function values f(r, theta) as 2D array of shape (len(x_vals), len(z_vals))
    """
    # Create meshgrid of polar coordinates
    Theta, R = np.meshgrid(x_vals, z_vals, indexing='ij')

    # Convert to Cartesian coordinates
    X = R * np.cos(Theta*np.pi)
    Y = R * np.sin(Theta*np.pi)
    Z = f_vals  # already in shape (len(x_vals), len(z_vals))

    # Plot using Plotly
    fig = go.Figure(data=[go.Surface(
        x=X, y=Y, z=Z,
        colorscale='Viridis',
        showscale=True
    )])

    fig.update_layout(
        title='Polar Surface Plot: f(r, θ)',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='f(r, θ)'
        ),
        autosize=True
    )

    fig.show()

x_vals_ds = x_vals_new[::2]
z_vals_ds = z_vals_new[::2]
f_vals_ds = f_vals_new[::2, ::2]  # downsample both axes

plot_polar_surface(x_vals_ds, z_vals_ds, f_vals_ds)


In [ ]:
np.min(x_vals_ds)

In [ ]:

def plot_polar_surface_r_lt(x_vals, z_vals, f_vals, r_max=0.4, eps=1e-12):
    """
    Plots f(r, theta) as a 3D surface using polar coordinates.
    x_vals: radial coordinates (r)
    z_vals: angular coordinates (theta, in radians)
    f_vals: function values f(r, theta) as 2D array of shape (len(x_vals), len(z_vals))
    """
    # Create meshgrid of polar coordinates
    Theta, R = np.meshgrid(x_vals, z_vals, indexing='ij')

    # Convert to Cartesian coordinates
    X = R * np.cos(Theta*np.pi)
    Y = R * np.sin(Theta*np.pi)
    Z = f_vals  # already in shape (len(x_vals), len(z_vals))

    # Mask values where r >= r_max
    Z[R >= r_max] = np.nan

    # Rescale axes
    X = X / r_max
    Y = Y / r_max

    # Plot using Plotly
    fig = go.Figure(data=[go.Surface(
        x=X, y=Y, z=Z,
        colorscale='Viridis',
        showscale=True
    )])

    fig.update_layout(
        title='Polar Surface Plot: f(r, θ)',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='f(r, θ)'
        ),
        autosize=True
    )
    
    fig.add_trace(go.Surface(
    x=X, y=Y, z=Z,
    colorscale='Viridis',
    showscale=True,
    contours=dict(
        z=dict(show=True, start=0, end=0.01, size=1e-1, color='red', width=6)
    ),
    name='f(θ, r)',
    opacity=0.95
    ))




    fig.show()

In [ ]:
plot_polar_surface_r_lt(x_vals_ds, z_vals_ds, f_vals_ds, r_max=0.4)
